In [42]:
import ollama
import numpy as np
import time
import os
import json
from numpy.linalg import norm

In [12]:
def parse_file(filename): 
    with open(filename, encoding = 'utf-8-sig') as f:
        paragraphs = []
        buffer = []

        for line in f.readlines():
            line = line.strip()
            if line:
                buffer.append(line)
            elif len(buffer):
                paragraphs.append(" ".join(buffer))
                buffer = []
                
        if len(buffer):
                paragraphs.append(" ".join(buffer))
                buffer = []

    return paragraphs

In [33]:
def get_embeddings(filename, modelname, chunks):
    if (embeddings := load_embeddings(filename)) is not False:
        return embeddings
    
    embeddings = [
        ollama.embeddings(model = modelname, prompt = chunk)['embedding'] 
        for chunk in chunks
    ]
    
    save_embeddings(filename, embeddings)

    return embeddings

In [29]:
def save_embeddings(filename, embeddings):
    if not os.path.exists('embeddings'):
        os.makedirs('embeddings')
    with open(f"embeddings/{filename}.json", "w") as f:
        json.dump(embeddings, f)

In [32]:
def load_embeddings(filename):
    if not os.path.exists(f"embeddings/{filename}.json"):
        return False
    with open(f"embeddings/{filename}.json", "r") as f:
        return json.load(f)

In [45]:
def find(needle, haystack):
    needle_norm = norm(needle)
    similarity_scores = [
        np.dot(needle, item) / (needle_norm * norm(item)) for item in haystack
    ]
    
    return sorted(zip(similarity_scores, range(len(haystack))), reverse = True)   

In [35]:
modelname = 'llama3'

In [25]:
paragraphs = parse_file('buddhism.txt')

In [34]:
start = time.perf_counter()
embeddings = get_embeddings('buddhism', modelname, paragraphs)
print(time.perf_counter() - start)

4.032627400010824


In [56]:
prompt = input("ask me: ")
prompt_embedding = ollama.embeddings(model = modelname, prompt = prompt)['embedding'] 

most_similar = find(prompt_embedding, embeddings)[:5]
for item in most_similar:
    print(item[0], paragraphs[item[1]])

SYSTEM_PROMPT = """
You are a reading assistant who answers questions based on snippets of text provided in the context. 
Answer only using the context provided. Answer shoul contain at most 5 sentences.
If unsure - just say you don't know.

Context:
"""

response = ollama.chat(model = modelname, messages = [
    {'role': 'system', 
     'content': SYSTEM_PROMPT + "\n".join(paragraphs[item[1]] for item in most_similar)},
    {'role': 'user', 
     'content': prompt}
])

print('===========================')
print(response['message']['content'])

ask me:  core beliefs


0.4792955726420998 124 Theory of Reincarnation Explained.
0.4726495725063703 19 Nietzsche: Who He Was and What He Stood For.
0.46757094532527166 Shakespeare's Plays
0.4657541745560094 The Essence of Buddhism
0.46500180876196656 347 A Guide to Stoicism.
I'm not sure which text is most relevant to "core beliefs". However, based on the provided context, I can tell you that Nietzsche was a philosopher who had some core beliefs about the will to power and the death of God. Similarly, Buddhism has some core principles such as the Four Noble Truths and the Eightfold Path.
